In [1]:
# Cell 1: Setup and Google Drive Mounting

import torch
import os

# 1. Verify GPU is available
if torch.cuda.is_available():
    print("GPU is available and enabled!")
    device = torch.device('cuda')
else:
    print("GPU is not available, running on CPU. Consider enabling GPU in Runtime settings.")
    device = torch.device('cpu')

# 2. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("\nGoogle Drive mounted successfully.")

GPU is available and enabled!
Mounted at /content/drive

Google Drive mounted successfully.


In [2]:
# Cell 2: Define Paths and Unzip Dataset

# --- IMPORTANT: Define the paths based on your Google Drive structure ---
# This is the folder you created in your Google Drive
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Colab_IntruderDetection/'

# Path to the uploaded ZIP file on your Drive
ZIP_FILE_PATH = os.path.join(DRIVE_PROJECT_PATH, 'INRIAPerson.zip')

# Path where the dataset will be extracted in Colab's temporary storage (for speed)
DATASET_EXTRACT_PATH = '/content/INRIAPerson'

# Path where the final trained model will be saved on your Google Drive (for persistence)
MODEL_SAVE_PATH = os.path.join(DRIVE_PROJECT_PATH, 'person_detector_model.pth')


# --- Unzip the dataset ---
# We check if the folder already exists to avoid unzipping again on re-running the cell
if not os.path.exists(DATASET_EXTRACT_PATH):
    print(f"Extracting dataset from '{ZIP_FILE_PATH}'...")
    # The '-q' flag makes the output quiet
    !unzip -q "{ZIP_FILE_PATH}" -d "/content/"
    print(f"Dataset extracted to '{DATASET_EXTRACT_PATH}'.")
else:
    print(f"Dataset already extracted at '{DATASET_EXTRACT_PATH}'.")

Extracting dataset from '/content/drive/MyDrive/Colab_IntruderDetection/INRIAPerson.zip'...
Dataset extracted to '/content/INRIAPerson'.


In [3]:
# Cell 3: Main Training Script

import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# ==============================================================================
# 1. DATASET LOADER (CONFIGURED FOR INRIA DATASET STRUCTURE)
# ==============================================================================
class PersonDetectionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        person_dir = os.path.join(root_dir, 'pos')
        if os.path.exists(person_dir):
            for img_name in os.listdir(person_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.images.append(os.path.join(person_dir, img_name))
                    self.labels.append(1)

        no_person_dir = os.path.join(root_dir, 'neg')
        if os.path.exists(no_person_dir):
            for img_name in os.listdir(no_person_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.images.append(os.path.join(no_person_dir, img_name))
                    self.labels.append(0)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Warning: Could not load image {image_path}. Error: {e}")
            return torch.randn(3, 224, 224), torch.tensor(0)

        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

# ==============================================================================
# 2. DEEP LEARNING MODEL (PERSON DETECTOR)
# ==============================================================================
class PersonDetector(nn.Module):
    def __init__(self, num_classes=2):
        super(PersonDetector, self).__init__()
        self.backbone = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
        for param in list(self.backbone.parameters())[:-10]:
            param.requires_grad = False
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.backbone(x)

# ==============================================================================
# 3. MODEL TRAINING FUNCTION
# ==============================================================================
def train_model(model, train_loader, val_loader, device, num_epochs=15):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    for epoch in range(num_epochs):
        print(f"--- Epoch {epoch+1}/{num_epochs} ---")
        model.train()
        running_loss = 0.0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * data.size(0)

        avg_train_loss = running_loss / len(train_loader.dataset)

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        val_accuracy = 100 * correct / total
        print(f'Train Loss: {avg_train_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%')
        scheduler.step()

    print("\nTraining complete.")
    return model

# ==============================================================================
# 4. SCRIPT EXECUTION
# ==============================================================================
# --- Configuration ---
NUM_EPOCHS = 15
BATCH_SIZE = 32

# --- Data Preparation ---
print("Preparing dataset...")
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Use the extracted path for the dataset
train_dir = os.path.join(DATASET_EXTRACT_PATH, 'Train')
test_dir = os.path.join(DATASET_EXTRACT_PATH, 'Test')

train_dataset = PersonDetectionDataset(root_dir=train_dir, transform=transform_train)
val_dataset = PersonDetectionDataset(root_dir=test_dir, transform=transform_val)

print(f"Found {len(train_dataset)} training images and {len(val_dataset)} validation images.")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

# --- Model Training ---
print("\nInitializing and training the model...")
model = PersonDetector(num_classes=2).to(device)
trained_model = train_model(model, train_loader, val_loader, device, num_epochs=NUM_EPOCHS)

# --- Save the Model ---
print(f"Saving trained model to {MODEL_SAVE_PATH}")
torch.save(trained_model.state_dict(), MODEL_SAVE_PATH)
print("Model saved successfully to your Google Drive.")

Preparing dataset...
Found 1832 training images and 741 validation images.

Initializing and training the model...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 205MB/s]


--- Epoch 1/15 ---
Train Loss: 0.1257 | Val Accuracy: 95.55%
--- Epoch 2/15 ---
Train Loss: 0.0636 | Val Accuracy: 97.03%
--- Epoch 3/15 ---
Train Loss: 0.0204 | Val Accuracy: 96.09%
--- Epoch 4/15 ---
Train Loss: 0.0352 | Val Accuracy: 96.49%
--- Epoch 5/15 ---
Train Loss: 0.0415 | Val Accuracy: 95.55%
--- Epoch 6/15 ---
Train Loss: 0.0232 | Val Accuracy: 95.28%
--- Epoch 7/15 ---
Train Loss: 0.0242 | Val Accuracy: 94.06%
--- Epoch 8/15 ---
Train Loss: 0.0189 | Val Accuracy: 97.30%
--- Epoch 9/15 ---
Train Loss: 0.0118 | Val Accuracy: 97.57%
--- Epoch 10/15 ---
Train Loss: 0.0226 | Val Accuracy: 98.25%
--- Epoch 11/15 ---
Train Loss: 0.0151 | Val Accuracy: 97.71%
--- Epoch 12/15 ---
Train Loss: 0.0112 | Val Accuracy: 97.98%
--- Epoch 13/15 ---
Train Loss: 0.0054 | Val Accuracy: 98.38%
--- Epoch 14/15 ---
Train Loss: 0.0147 | Val Accuracy: 97.44%
--- Epoch 15/15 ---
Train Loss: 0.0044 | Val Accuracy: 97.57%

Training complete.
Saving trained model to /content/drive/MyDrive/Colab_Intrud